In [5]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom, binary_dilation
from scipy.io import loadmat
import h5py
import nibabel as nib
import re

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from noise_ceiling import (
    compute_ncsnr,
    compute_nc,
    group_repetitions
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
dataset_root = Path('D:\Datasets')

In [7]:
tc2see_version = 3 # [1, 2]
dataset_path = dataset_root / f"TC2See_v{tc2see_version}"
derivatives_path = dataset_path / "derivatives_TC2See_prdgm"
num_runs = 6

task = "bird"
subject = '03' # ['03', '04']
glm_run = 'level1_native_singletrial'
singletrial = True

# Initialize BIDSLayouts for querying files.
dataset_layout = BIDSLayout(dataset_path / 'TC2See_prdgm')
#betas_path = derivatives_path / 'spm' / f"sub-{subject}" / glm_run
betas_path = derivatives_path / 'spm' / glm_run

In [8]:
write_file = True

spm = loadmat(betas_path / 'SPM.mat')
descrip = spm['SPM'][0][0]['Vbeta']['descrip'][0]
stimulus_ids = []
for i in range(len(descrip)):
    if singletrial:
        name = re.search ('\d+_([a-z_]+)_\d+', descrip[i][0])
    else:
        name = re.search('\) (\w+)\*', descrip[i][0])

    if name != None:
        stimulus_ids.append(name.groups(1)[0])

print(stimulus_ids)

# write file
if write_file:
    with open(betas_path / 'image_labels.txt', 'w') as f:
        for name in stimulus_ids:
            f.write(name + '\n')


['anchor', 'antelope', 'alpaca', 'aardvark', 'badger', 'beanbag', 'bag', 'basketball', 'air_pump', 'bear', 'anvil', 'biscuit', 'air_pump', 'bag', 'bear', 'beanbag', 'badger', 'anchor', 'biscuit', 'basketball', 'anvil', 'antelope', 'alpaca', 'aardvark', 'badger', 'bag', 'alpaca', 'aardvark', 'bear', 'beanbag', 'anvil', 'anchor', 'air_pump', 'basketball', 'antelope', 'biscuit', 'badger', 'alpaca', 'beanbag', 'bag', 'air_pump', 'antelope', 'aardvark', 'bear', 'anchor', 'anvil', 'basketball', 'biscuit', 'basketball', 'anvil', 'bag', 'beanbag', 'aardvark', 'air_pump', 'biscuit', 'badger', 'antelope', 'bear', 'anchor', 'alpaca', 'anvil', 'badger', 'antelope', 'basketball', 'alpaca', 'biscuit', 'air_pump', 'bag', 'beanbag', 'aardvark', 'anchor', 'bear', 'cow', 'brush', 'bison', 'dartboard', 'camel', 'dagger', 'cheesecake', 'cat', 'dalmatian', 'bull', 'box', 'chick', 'dartboard', 'box', 'bison', 'cat', 'bull', 'brush', 'dagger', 'camel', 'chick', 'dalmatian', 'cheesecake', 'cow', 'brush', 'chi

In [9]:
# load mask
mask = nib.load(betas_path / 'mask.nii').get_fdata().astype(bool)
print(mask.shape)

# load betas images
betas = []
num_betas = 432 if singletrial else 72

for fnum in range(1, num_betas+1):
    img = nib.load(betas_path / f"beta_{fnum:04}.nii").get_fdata()
    img = img[mask]
    betas.append(img)

betas = np.array(betas)
print(betas.shape)

(102, 102, 64)
(432, 125506)


In [10]:
betas_normalized = (betas - betas.mean(axis=0, keepdims=True)) /  betas.std(axis=0, keepdims=True)

In [ ]:
# detrending

num_betas = betas.shape[0]
trend_coeffs = np.stack([np.arange(num_betas), np.ones(shape=num_betas)], axis=1)
betas_trend = np.linalg.lstsq(trend_coeffs, betas, rcond=None)[0]
betas_predicted = trend_coeffs @ betas_trend
betas_detrend = betas - betas_predicted
betas_detrend_std = betas_detrend.std(axis=0)

print(betas.mean(), betas_detrend.mean())
print(betas.std(axis=0))
print(betas_detrend.std(axis=0))

# linear_trend normalization
images = range(1, num_betas + 1) # treat the 72 betas as a 'run'
trend_coeffs = np.stack([images, np.ones_like(images)], axis=1).astype(float)
predicted_betas = trend_coeffs @ betas_trend
betas_normalized = (betas - predicted_betas) / betas_detrend_std

In [ ]:
# view mean betas
print(betas.shape)
print(mask.shape)

betas_mean = betas_normalized.mean(axis=0)
print(min(betas_mean), max(betas_mean))

mi = min(betas_mean)
ma = max(betas_mean)

brain_volume = np.zeros_like(mask, dtype=float)
brain_volume[mask] = betas_mean
D = brain_volume.shape[2]
@interact(d=(0, D-1), original=True)
def show(d):
    plt.figure(figsize=(12, 12))
    plt.imshow(brain_volume[:, :, d], cmap='bwr', vmin=mi, vmax=ma)

In [11]:
_, stimulus_ids2 = np.unique(stimulus_ids, return_inverse=True)
print(len(np.unique(stimulus_ids)))

36


In [16]:
run_mask = np.array([1,1,1,1,1,1], dtype=bool).repeat(72)

In [17]:
grouped_repetitions = []
for i in range(2, 15):
    x = group_repetitions(stimulus_ids2[run_mask], num_repetitions=i)
    if x is not None:
        grouped_repetitions.append(x)
        
print([x.shape for x in grouped_repetitions])
print(grouped_repetitions)

[(36, 12)]
[array([[  3,  23,  27,  42,  52,  69, 222, 228, 249, 254, 264, 277],
       [  8,  12,  32,  40,  53,  66, 223, 234, 243, 256, 270, 280],
       [  2,  22,  26,  37,  59,  64, 219, 235, 240, 262, 275, 282],
       [  0,  17,  31,  44,  58,  70, 227, 233, 245, 252, 273, 279],
       [  1,  21,  34,  41,  56,  62, 221, 236, 244, 261, 265, 283],
       [ 10,  20,  30,  45,  49,  60, 216, 231, 246, 259, 266, 276],
       [  4,  16,  24,  36,  55,  61, 225, 229, 250, 257, 269, 281],
       [  6,  13,  25,  39,  50,  67, 226, 239, 248, 260, 271, 286],
       [  7,  19,  33,  46,  48,  63, 217, 238, 242, 253, 268, 284],
       [  5,  15,  29,  38,  51,  68, 218, 230, 251, 263, 267, 285],
       [  9,  14,  28,  43,  57,  71, 224, 232, 241, 255, 274, 278],
       [ 11,  18,  35,  47,  54,  65, 220, 237, 247, 258, 272, 287],
       [ 74,  86,  99, 110, 131, 138, 290, 305, 317, 324, 338, 353],
       [ 82,  85, 103, 114, 124, 141, 296, 301, 318, 333, 343, 359],
       [ 73,  89,  96,

In [18]:
ncsnr = compute_ncsnr(betas_normalized[run_mask], grouped_repetitions)
nc = compute_nc(ncsnr, num_averages=1)
nc_volume = np.zeros_like(mask, dtype=float)
nc_volume[mask] = nc

In [ ]:
print(mask.shape)
print(nc_volume.shape)
print(nc.shape)
print(79*95*79)
print(mask.sum())
print(betas.shape)
print(grouped_repetitions)

In [ ]:
print((nc_volume[:, :, 5:-5] ).max())
print((nc_volume[:, :, 5:-5] ).min())

In [19]:
# plot noise ceiling

D = nc_volume.shape[2]
@interact(d=(0, D-1), original=True)
def show(d):
    plt.figure(figsize=(12, 12))
    plt.imshow(nc_volume[:, :, d], cmap='jet', vmin=0, vmax=100,)

interactive(children=(IntSlider(value=31, description='d', max=63), Output()), _dom_classes=('widget-interact'…

In [ ]:
# save noise ceiling

fname = betas_path / 'noise_ceiling__run-0-1-3-4.nii.gz'
image = nib.Nifti1Image(nc_volume, np.eye(4))
nib.save(image, fname)